<a href="https://colab.research.google.com/github/iaehistoriaUFSC/Repositorio_UFSC/blob/main/Extracao_de_dados/via_webscraping/coleta_link_por_colecao/repositorio_geral/2%20-%20Segunda%20metade%20das%20colec%CC%A7o%CC%83es%20-%20%5B1%20de%202%20executores%5D%20-%20Aguardando%20execuc%CC%A7a%CC%83o/Coleta_de_Links_das_cole%C3%A7%C3%B5es_Web_Scrapping_Reposit%C3%B3rio_UFSC_exec1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Preparação do ambiente

**Execução 1** - baixando e importanto bibliotecas utilizadas ao decorrer da execução do programa

In [ ]:
# Instalando as bibliotecas que serão utilizadas
try:
  !pip install requests
  !pip install beautifulsoup4
  !pip install pandas
  !pip install regex
  !pip install urllib3
except: # Caso ocorra algum erro ao instalar os pacotes/bibliotecas seremos notificados com uma mensagem na tela de output
  print('\nErro ao instalar pacotes/bibliotecas.')
  print('Descrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'\n')
else:
  from google.colab import drive
  from google.colab import output
  import requests
  from bs4 import BeautifulSoup
  import pandas as pd
  import time
  import re
  from urllib import request
  import os
  import sys
  execucao = True # Variável que armazenará o sucesso no download das bibliotecas
  output.clear()
  print('='*100)
  print('Bibliotecas carregadas!\nPode executar o código abaixo.')
  print('='*100)

**Execução 2** - Conectando ambiente do Google Colab ao Google Drive

In [ ]:
try:
  drive.mount('/content/drive')
except:
  print('\nErro ao "sincronizar" Google Drive com este ambiente do Google Colab.\nExecute a célula novamente ou tente "montar" o Drive manualmente no botão de "Montar Drive" no menu lateral esquerdo.')
  print('Descrição do erro --> '+str(sys.exc_info()[0].__name__)+":",str(sys.exc_info()[1])+'\n')
  execucao = False
else:
  print('='*100)
  print('Drive conectado com sucesso.\nPode prosseguir na execução das próximas células.')
  print('='*100)

**Execução 3** - Definindo variáveis e funções que serão utilizadas ao decorrer do programa

In [ ]:
if execucao:


  headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
                (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"}
  timeout = 30

  lista_de_falhas = {'Avisos':[],'Erros':[]}


  try:
    def CriaDiretorio(caminho : str):
      try:
        os.makedirs(caminho)
        time.sleep(1)
        if os.path.isdir(caminho):
          print(f'Pasta {caminho} criada com sucesso.')
          return True
        else:
          print(f'Tentando criar pasta {caminho} novamente.')
          time.sleep(3)
          if os.path.isdir(caminho):
            print(f'Pasta {caminho} criada com sucesso.')
            return True
          else:
            print(f'Pasta {caminho} não foi carregada corretamente.')
            return False
      except:
        print(f'Erro ao criar diretório {caminho}.\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'\n')
        return False

    def ColetaHTML(link_pagina : str):
      try:
        site = requests.get(link_pagina,headers=headers,timeout=timeout)
        if site.status_code == 200:
          soup = BeautifulSoup(site.content, 'html.parser')
          return True, soup
        else:
          print(f'Problema ao coletar HTML da página {link_pagina}.\nsite.status_code != 200')
          return False, ''
      except:
        print(f'Problema ao coletar HTML da página {link_pagina}.\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'.')
        return False, ''


    def ColetaNomesLinksColecoesDoHTML(soup):
      lista_nomes_colecoes = []
      lista_links_colecoes = []

      campo_colecoes = soup.find('div', class_='ds-static-div secondary')
      if campo_colecoes != None:
        colecoes = campo_colecoes.find_all('li')
        if len(colecoes) == 0:
          print('\nNão foi possível encontrar os elementos das coleções corretamente.')
          lista_de_falhas['Erros'].append('Não foi possível encontrar os elementos das coleções corretamente.')
          return False, '', ''
        else:
          print('Quantidade de coleções avistadas:',len(colecoes))

          tamanho_total = len(colecoes)
          metade_tamanho_total = int(tamanho_total/2)

          for indice, colecao in enumerate(colecoes[metade_tamanho_total:]):
            nome_e_link_colecao = colecao.find('a', href=re.compile('/handle'))
            try:
              nome_colecao = nome_e_link_colecao.text

              lista_nomes_colecoes.append(nome_colecao)

              link_colecao = nome_e_link_colecao['href']
              link_colecao = 'https://repositorio.ufsc.br'+link_colecao

              lista_links_colecoes.append(link_colecao)
            except:
              print(f'\nErro ao encontrar o nome e link da coleção número {indice} da lista.\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+":",str(sys.exc_info()[1])+'\n')
              lista_de_falhas['Avisos'].append(f'Erro ao encontrar o nome e link da coleção número {indice} da lista.\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'\n')
            # break # Só vai pegar a primeira coleção

            # Limitando número de coleções que serão visitadas
            # limite_colecoes = 3
            # if indice >= limite_colecoes: # quebra da iteração
            #   print(f'\nLIMITE DE COLEÇÕES ({limite_colecoes+1}) ATINGIDO.')
            #   break

          if lista_nomes_colecoes and lista_links_colecoes:
            return True, lista_nomes_colecoes, lista_links_colecoes
          else:
            return False, [], []

      else:
        print('Campo de coleções não identificado corretamente na página. "campo_colecoes == None"!')
        return False,'',''

    def ColetaLinksTrabalhosDoHTML(soup_colecao,link_principal_colecao):
      lista_links_trabalhos = []
      numero_paginas = soup_colecao.find('div', attrs={"class":'pagination-masked top'})
      if numero_paginas == None:
        print('\nNão foi possível achar número de páginas corretamente.\n(numero_paginas == None).')
        lista_de_falhas["Avisos"].append(f'Não foi possível achar número de páginas corretamente da coleção {dic["Colecoes"][i]} --> (numero_paginas == None).')
        return False, []
      else:
        paginas_links = numero_paginas.find('ul',class_='pagination-links')
        if paginas_links == None:
          print('\nNão foi possível achar número de páginas corretamente.\n(paginas_links == None)')
          lista_de_falhas["Avisos"].append(f'Não foi possível achar número de páginas corretamente da coleção {dic["Colecoes"][i]} --> (paginas_links == None).')
          return False, []
        else:
          links_pagina = paginas_links.find_all('li')
          if links_pagina == None:
            print('\nNão foi possível achar número de páginas corretamente.\n(links_pagina == None)')
            lista_de_falhas["Avisos"].append(f'Não foi possível achar número de páginas corretamente da coleção {dic["Colecoes"][i]} --> (links_pagina == None).')
            return False, []
          else:
            try:
              numero_paginas = int(links_pagina[len(links_pagina)-1].find('a').text)
            except:
              print('\nNúmero de páginas não pode ser convertido para inteiro, logo foi setado como 1.\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'\n')
              numero_paginas = 1
              lista_de_falhas["Avisos"].append(f'Número de páginas da coleção {dic["Colecoes"][i]} não pode ser convertido para inteiro, logo foi setado como 1\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'\n')
            print('\nNúmero total de páginas:',numero_paginas)


            links = soup_colecao.find_all('li', class_=re.compile('ds-artifact-item clearfix'))
            if len(links) == 0:
              print('\nNão foi possível encontrar corretamente os links para os trabalhos.')
              lista_de_falhas["Avisos"].append(f'Não foi possível encontrar/captar corretamente os links para os trabalhos da coleção {dic["Colecoes"][i]}.')
              return False, []
            else:
              pagina = 1

              while(pagina <= numero_paginas):
                print('\nIndo para página:',pagina)
                link_p_trabalhos = link_principal_colecao + f'/discover?rpp=50&etal=0&group_by=none&page={pagina}'
      #                                  MUDAR O RPP = 50 PARA RPP = 100  QUANDO FOR FAZER NO REPOSITÓRIO TODO (falar com TI da BU antes) !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
                status_conteudo_HTML_links_trabalho, soup_links_trabalho = ColetaHTML(link_p_trabalhos)
                if status_conteudo_HTML_links_trabalho:
                  links = soup_links_trabalho.find_all('li', class_=re.compile('ds-artifact-item clearfix'))
                  if len(links) == 0:
                    print(f'\nNão foi possível encontrar corretamente os links dos resultados da coleção {dic["Colecoes"][i]} no link: "{link}".')
                    lista_de_falhas['Avisos'].append(f'Não foi possível encontrar/coletar corretamente os links dos resultados da coleção {dic["Colecoes"][i]} no link: "{link}".')
                  else:
                    for links_trabalho in links:
                      link_trabalho = links_trabalho.find('a', href=re.compile('/handle'))
                      if link_trabalho != None:
                        link_trabalho = link_trabalho['href']
                        link_trabalho = 'https://repositorio.ufsc.br'+link_trabalho
                        lista_links_trabalhos.append(link_trabalho)
                      else:
                        print(f'\nLink do trabalho não foi encontrado corretamente. Coleção: {dic["Colecoes"][i]}, link: "{link}".')
                        lista_de_falhas['Avisos'].append(f'Link do trabalho não foi encontrado corretamente. Coleção: {dic["Colecoes"][i]}, link: "{link}".')

                  # # Limitando a busca (primeiramente não vamos passar por todos os resultados)
                  # limite_pagina = 2
                  # if pagina >= limite_pagina:
                  #   print(f'\nLIMITE DE PÁGINAS ({limite_pagina}) ALCANÇADO.')
                  #   break
                  # else:
                  #   pagina += 1
                  pagina += 1
              return True, lista_links_trabalhos

  except:
    print('Erro ao criar funções.\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'\n')
  else:
    print('='*100)
    print('Variáveis para requisições e funções de criação de diretório criadas com sucesso.\nPode dar continuidade na execução das células abaixo.')
    print('='*100)
else:
  print('\nFalha no carregamento das bibliotecas ou sincronização com Drive. Tente executar as primeiras células de códigos novamente.')

#Execução das ações, de fato

**Execução 4** - Criação de pasta "Coleções Repositorio UFSC" e sub-pasta "Links dos trabalhos - Repositorio UFSC" para armazenamento dos primeiros dados (links das coleções)

In [ ]:
if execucao:
  try:
    # CriaDiretorio("/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Extração de Dados/via Web Scraping/Resultados/Coleções Repositorio UFSC/Links dos trabalhos - Repositorio UFSC")
    CriaDiretorio("/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Extração de Dados/via Web Scraping/Otimizações/Resultados/Coleções Repositorio UFSC/Links dos trabalhos - Repositorio UFSC")
  except:
    print('\nErro ao criar pasta no Drive com o código do Google Colab.\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+":",str(sys.exc_info()[1])+'\n')
    execucao = False
  else:
    print('='*100)
    print('Pastas de trabalhos "Coleções Repositorio UFSC" e "Links dos trabalhos - Repositorio UFSC" criadas com sucesso!\nPode continuar e executar os códigos abaixo.')
    print('='*100)
else:
  print('\nFalha no carregamento das bibliotecas. Tente executar a primeira célula de códigos novamente.\n')

**Execução 5** - Preenchimento de dados (links) das coleções e dos trabalhos de cada coleção

In [ ]:
if execucao:

  start = time.time()


  link = 'https://repositorio.ufsc.br/handle/123456789/74645'


  status_conteudo_HTML, soup_repositorio = ColetaHTML(link)
  if status_conteudo_HTML:
    numero_total_de_trabalhos = 0
    dic = {'Colecoes':[],'Links colecoes':[]}
    lista_links_trabalhos = []

    status_info_HTML,lista_nomes_colecoes,lista_links_colecoes = ColetaNomesLinksColecoesDoHTML(soup_repositorio)
    if status_info_HTML:
      dic['Colecoes'] = lista_nomes_colecoes
      dic['Links colecoes'] = lista_links_colecoes
      print('\nNúmero de coleções encontradas:',len(dic['Links colecoes']))
      num_colecao = 0


      for i in range(len(dic['Links colecoes'])):
        dic_trabalho = {'Links trabalhos':[]}
        num_colecao +=1
        link_principal_colecao = dic['Links colecoes'][i]

        link_colecao_atual = link_principal_colecao + '/discover?rpp=50&etal=0&group_by=none&page=1'
    #                                  MUDAR O RPP = 50 PARA RPP = 100  QUANDO FOR FAZER NO REPOSITÓRIO TODO (falar com TI da BU antes) !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

        status_conteudo_HTML, soup_colecao = ColetaHTML(link_colecao_atual)
        if status_conteudo_HTML:
          status_links_trabalhos, lista_links_trabalhos = ColetaLinksTrabalhosDoHTML(soup_colecao,link_principal_colecao)
          if status_links_trabalhos:
            dic_trabalho['Links trabalhos'] = lista_links_trabalhos
            numero_total_de_trabalhos += len(lista_links_trabalhos)
            print('\nNúmero de links coletados:',len(dic_trabalho["Links trabalhos"]))

            try:
              df = pd.DataFrame(dic_trabalho)
            except:
              print('Problemas ao transformar o dicionário "dic_trabalho" num dataframe pandas.')
            else:
              if '/' in dic["Colecoes"][i]:
                index = dic["Colecoes"][i].find('/')
                dic["Colecoes"][i] = dic["Colecoes"][i][:index]+'_'+dic["Colecoes"][i][index+1:]
              if ':' in dic["Colecoes"][i]:
                index = dic["Colecoes"][i].find(':')
                dic["Colecoes"][i] = dic["Colecoes"][i][:index]+' '+dic["Colecoes"][i][index+1:]
              try:
                df.to_csv(f'/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Extração de Dados/via Web Scraping/Otimizações/Resultados/Coleções Repositorio UFSC/Links dos trabalhos - Repositorio UFSC/{dic["Colecoes"][i]}_Links_dos_Trabalhos.csv',index=False,encoding='utf-8',sep=',')
              except:
                print(f'\nErro ao fazer download do arquivo .csv para coleção {dic["Colecoes"][i]}.\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+":",str(sys.exc_info()[1])+'\n')
                lista_de_falhas['Erros'].append(f'Erro ao fazer download do arquivo .csv para coleção {dic["Colecoes"][i]}.\nDescrição do erro --> '+str(sys.exc_info()[0].__name__)+": "+str(sys.exc_info()[1])+'\n')
              else:
                print(f'\nDownload do arquivo .csv da coleção {dic["Colecoes"][i]} feito com sucesso!\n')


        # # Código abaixo comentado, pois era apenas para fins de limitar o número de coleções que serão visitadas
        # num_colecao_maxima = 4
        # if num_colecao >= num_colecao_maxima-1:
        #   print(f'\nParou na {num_colecao}a coleção.')
        #   break

  print('\n')
  print('='*100)
  print('Programa executado com sucesso!\nNúmero total de links de trabalhos coletados:',numero_total_de_trabalhos)
  print(f'Número de Coleções visitadas:',len(dic["Links colecoes"]))
  print('='*100)
  print('\n')
  print('='*100)
  print('Número de avisos gerados:',len(lista_de_falhas["Avisos"]))
  print('Número de erros gerados:',len(lista_de_falhas["Erros"]))
  print('='*100)


  if (len(lista_de_falhas['Avisos']) > 0):
    file_path = '/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Extração de Dados/via Web Scraping/Otimizações/Resultados/Coleções Repositorio UFSC/Links dos trabalhos - Repositorio UFSC/AVISOS primeira metade.txt'
    texto = 'AVISOS:\n\n'
    for i in range(len(lista_de_falhas['Avisos'])):
      texto = texto + '\n' + lista_de_falhas['Avisos'][i]
    with open(file_path, 'w') as f:
      f.write(texto)
      print('\nArquivo de avisos salvo com sucesso.\nVerifique-o!')
      f.close()
  if (len(lista_de_falhas['Erros']) > 0):
    file_path = '/content/drive/MyDrive/Programa - Repositório Institucional UFSC/Extração de Dados/via Web Scraping/Otimizações/Resultados/Coleções Repositorio UFSC/Links dos trabalhos - Repositorio UFSC/ERROS primeira metade.txt'
    texto = 'ERROS:\n\n'
    for i in range(len(lista_de_falhas['Erros'])):
      texto = texto + '\n' + lista_de_falhas['Erros'][i]
    with open(file_path, 'w') as f:
      f.write(texto)
      print('\nArquivo de erros salvo com sucesso.\nVerifique-o!')
      f.close()

  end = time.time()

  duracao = end - start
  print('\n')
  print('='*100)
  print('Duração total da primeira execução:',round(duracao,2))
  print('='*100)
else:
  print('\nFalha na execução das células anteriores. Volte, leia as saídas das células anteriores e tente executá-las novamente.\n')